
Welcome to this tutorial on installing and using the latest CorgiSim simulation tools. In this guide, we'll walk through how to set up and run simulations of an on-axis host star with off-axis companions for observations with the Roman Coronagraph Instrument (CGI).

 This tutorial is intended to help you get started quickly and to provide a clear overview of CorgiSim’s current infrastructure and workflow. It's especially geared toward those interested in contributing to the development and improvement of the code.


# Installation

Since **CorgiSim** is still under active development, we recommend cloning the **main** branch from the repository and installing it in **editable mode**. It's also best to work in a **virtual environment** to avoid conflicts with other packages.

## Step-by-step instructions

1. **Clone the repository:**


```
    git clone -b main https://github.com/roman-corgi/corgisim.git
    cd corgisim
```
2. **Install in editable mode:**

```
    pip install -e .
```

## Required dependencies
To install the required dependencies, run:
```
    pip install -r requirements.txt
```
This will install all Python packages available via `pip`, including `synphot`.  
For more information on `synphot`, refer to its [documentation](https://synphot.readthedocs.io/en/latest/#installation-and-setup).

> ⚠️ Note: Some packages must be installed manually because they are not available on PyPI:

- **cgisim**: [https://sourceforge.net/projects/cgisim/](https://sourceforge.net/projects/cgisim/)
- **roman_preflight**: [https://sourceforge.net/projects/cgisim/files/](https://sourceforge.net/projects/cgisim/files/)
- **PROPER**: [https://sourceforge.net/projects/proper-library/](https://sourceforge.net/projects/proper-library/)

Detailed installation instructions for the required dependencies can be found in the README of Corgisim github repo: https://github.com/roman-corgi/corgisim/tree/main?tab=readme-ov-file



# Run the first simulation


In [ ]:
## import packages
from corgisim import scene
from corgisim import instrument
import matplotlib.pyplot as plt
import numpy as np
import proper
from corgisim import outputs
from astropy.io import fits

# import astropy.units as u

#### Step 0: Copying a Prescription File

To get started, you'll need a Roman CGI prescription file in your working folder. The `roman_preflight_proper` package provides a helper function to copy a default prescription file into your current working directory.         

Note: If the file is already present, you can skip this step. However, you'll need to repeat it whenever you switch to a new working directory.



In [ ]:
#First, import the module:
import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
# roman_preflight_proper.copy_here()

# off-axis PSF without mask? idk if this applies for 2D convolution

### Step 1: Define the Astrophysical Scene

Begin by specifying the astrophysical scene you want to simulate. This typically includes:

- A **host star**
- One or more **companions** (e.g., exoplanets or brown dwarfs)
- *(Future feature)* A 2D background scene, such as circumstellar disks — *not yet implemented*

> **Note:** As of now, only point sources (host stars and companions) are supported. 2D scene functionality will be added in future versions.

#### 🔹 Host Star

The following parameters are required for defining the host star:

1. **Spectral type** (e.g., `"G2V"`)
2. **V-band magnitude** (numeric value)
3. **Magnitude type** — currently, only **Vega magnitude** (`"vegamag"`) is supported

#### 🔹 Companions

You may define multiple companions. For each one, specify:

1. **V-band magnitude** brigtness of the companions  
2. **Magnitude type** — currently, only **Vega magnitude** (`"vegamag"`) is supported  
3. **position_x** — X-coordinate of the source position in milliarcseconds (mas), relative to the host star  
4. **position_y** — Y-coordinate of the source position in milliarcseconds (mas), relative to the host star  
For companion spectrum, currently, a flat spectrum will be generated based on the V-band magnitude

✅ Make sure to provide values in the expected format so the simulation can interpret the astrophysical inputs correctly.


In [ ]:
# --- Host Star Properties ---
Vmag = 8                            # V-band magnitude of the host star
sptype = 'G0V'                      # Spectral type of the host star
ref_flag = False                    # if the target is a reference star or not, default is False
host_star_properties = {'Vmag': Vmag,
                        'spectral_type': sptype,
                        'magtype': 'vegamag',
                        'ref_flag': False}

# --- Companion Properties ---
# Define two companions
mag_companion = [25, 25]           # List of magnitudes for each companion

# Define their positions relative to the host star, in milliarcseconds (mas)
# For reference: 1 λ/D at 550 nm with a 2.3 m telescope is ~49.3 mas
# We're placing them at a separation of 3 λ/D
dx = [3 * 49.3, -3 * 49.3]         # X positions in mas for each companion
dy = [3 * 49.3, -3 * 49.3]         # Y positions in mas for each companion

# Construct a list of dictionaries for all companion point sources
point_source_info = [
    {
        'Vmag': mag_companion[0],
        'magtype': 'vegamag',
        'position_x': dx[0],
        'position_y': dy[0]
    },
    {
        'Vmag': mag_companion[1],
        'magtype': 'vegamag',
        'position_x': dx[1],
        'position_y': dy[1]
    }
]


#### Adding 2D scene properties 

In [ ]:
# --- 2D scene properties ---
import corgisim.convolution as conv 
path = # Your path to the fits file 
fname_disk = path + 'eps_eri_su_34_2.fits' # Your disk file name
fname_prf = # the pre-computed prf cube

# settings used in the pre-computed prf cubes (see the tutorials below)
# Below is the command used to generate the example prf_cube (do not change it, unless it's needed)
radii_lamD = conv.build_radial_grid(
    inner_step      = 1,     # step inside IWA (λ/D)
    mid_step        = 1,     # step between IWA and OWA
    iwa             = 1,     # inner working angle (λ/D)
    owa             = 10,    # outer working angle (λ/D)
    outer_step = 1.
)
azimuths_deg = conv.build_azimuth_grid(step_deg=10)

# Note: The disk contrast is defined as the average surface brightness per spatial resolution element divided by the stellar flux. 
# The surface brightness is averaged over the disk regions (>50% of maximum disk flux) only, not the entire field of view (FoV).
disk_contrast = 16   # Delta Mag (unit:mag) 

twoD_scene_info = {
        'disk_model_path': fname_disk,
        'contrast': disk_contrast, 
        'prf_cube_path': fname_prf,
        'radii_lamD': radii_lamD,
        'azimuths_deg': azimuths_deg
    }

# --- Create the Astrophysical Scene ---
# This Scene object combines the host star and companion(s), and optionally a 2D scene (e.g., disk)
base_scene = scene.Scene(host_star_properties, point_source_info, twoD_scene_info)

# --- Access the generated disk spectrum ---
sp_disk = base_scene.twoD_scene_spectrum

# --- Access the generated stellar spectrum ---
sp_star = base_scene.stellar_spectrum
# --- Access the generated companion spectrum ---
sp_comp = base_scene.off_axis_source_spectrum 

The output of ```base_scene.stellar_spectrum``` is a ```SourceSpectrum``` object from ```synphot``` package. Details see https://synphot.readthedocs.io/en/latest/api/synphot.spectrum.SourceSpectrum.html#synphot.spectrum.SourceSpectrum

The output of ```base_scene.off_axis_source_spectrum```  is a list of ```SourceSpectrum``` object,  each corresponding to an input companion.

The default unit for wavelength is **Angstrom**, and the default unit for flux density is **photlam** (photons/s/cm^2/Angstrom). For units in ```synphot``` see: https://synphot.readthedocs.io/en/latest/synphot/units.html#synphot-flux-units

Let's plot the stellar spectrum to have a look. Currently, we only have blackbody spectrum, but will add more complated ones in future.


In [ ]:
ax=plt.subplot(111)
#sp.plot(ax=ax)
lambd = np.linspace(2000, 40000, 1000)
ax.plot(lambd , sp_star(lambd  ).value,label='corgisim stellar spectrum')
ax.set_xlabel('wavelength (A)')
ax.set_ylabel('flux density (photons/s/cm^2/A)')
plt.legend()

Let's then plot the spectrum of the first companion. Since we didn't provide a custom spectrum, it uses the default flat spectrum automatically generated based on its V-band magnitude.

In [ ]:
ax=plt.subplot(111)
#sp.plot(ax=ax)
lambd = np.linspace(2000, 40000, 1000)
ax.plot(lambd , sp_comp[0](lambd  ).value,label='corgisim companion spectrum')
ax.set_xlabel('wavelength (A)')
ax.set_ylabel('flux density (photons/s/cm^2/A)')
plt.legend()

In [ ]:
ax=plt.subplot(111)
#sp.plot(ax=ax)
lambd = np.linspace(2000, 40000, 1000)
ax.plot(lambd , sp_disk(lambd  ).value,label='corgisim disk spectrum')
ax.set_xlabel('wavelength (A)')
ax.set_ylabel('flux density (photons/s/cm^2/A)')
plt.legend()

#### Step 2: now you need to define the parameters for the telescope and instrument

The simulation mode is inherited from cgisim. In general, there are three supported modes, though currently only excam is implemented.

In [ ]:
# Simulation mode (currently only 'excam' is implemented)
# Options include:
#   - 'excam': direct imaging mode (returns intensity image)
#   - 'spec': spectral mode using the "spc-spec" coronagraph
#   - 'excam_efield': like 'excam', but returns electric field across wavelengths instead of intensity
cgi_mode = 'excam'


Choose the coronagraph type to use in the simulation. As of now, only hlc is implemented.

In [ ]:
# Coronagraph type
# Options (availability depends on implementation status):
#   - 'hlc', 'hlc_band1', 'hlc_band2', 'hlc_band3', 'hlc_band4'
#   - 'spc-spec_band2', 'spc-spec_band3'
#   - 'spc-wide_band1', 'spc-wide_band4'
#   - 'spc-mswc_band1', 'spc-mswc_band4'

cor_type = 'hlc'


Set bandpass filter to use with the selected coronagraph. The available options depend on the chosen **cor_type**.  
For example, if using `'hlc'` or `'hlc_band1'`, valid bandpasses include: `'1'`, `'1a'`, `'1b'`, `'1c'`, `'1_all'`.  
Refer to the table below for the valid bandpasses corresponding to each **cor_type**, as well as the definitions of the bandpasses. For more details on available bandpasses and how they’re defined, see the cgisim documentation: **cgisim_public_v4.0.pdf** You can donwload the cgisim_v4.0.zip from https://sourceforge.net/projects/cgisim/files/. The ZIP archive includes the documentation file.


| **cor_type**                                               | **Allowed Bandpasses**                                  |
|------------------------------------------------------------|----------------------------------------------------------|
| hlc or hlc_band1                                           | 1F, 1A, 1B, 1C, 1_ALL                                    |
| hlc_band2                                                  | 2F, 2A, 2B, 2C, 3A, 3B                                   |
| hlc_band3                                                  | 3F, 3A, 3B, 3C, 3D, 3E, 3G                               |
| hlc_band4                                                  | 4F, 4A, 4B, 4C                                           |
| spc-spec_band2                                             | 2F, 2A, 2B, 2C, 3A, 3B                                   |
| spc-spec_band2_rotated                                     | 2F, 2A, 2B, 2C, 3A, 3B                                   |
| spc-spec or spc-spec_band3                                 | 3F, 3A, 3B, 3C, 3D, 3E, 3G                               |
| spc-wide_band1                                             | 1F, 1A, 1B, 1C                                           |
| spc-wide or spc-wide_band4                                 | 4F, 4A, 4B, 4C                                           |
| spc-mswc_band1                                             | 1F, 1A, 1B, 1C                                           |
| spc-mswc or spc-mswc_band4                                 | 4F, 4A, 4B, 4C                                           |

| Bandpass Name | Central Wavelength λ<sub>c</sub> | FWHM Bandwidth Δλ/λ<sub>c</sub> |
|---------------|-------------------------------|------------------------------|
| 1F             | 575 nm                        | 10.1 %                       |
| 1A            | 556 nm                        | 3.5 %                        |
| 1B            | 575 nm                        | 3.3 %                        |
| 1C            | 594 nm                        | 3.2 %                        |
| 2F             | 660 nm                        | 17.0 %                       |
| 2A            | 615 nm                        | 3.6 %                        |
| 2B            | 638 nm                        | 2.8 %                        |
| 2C            | 656 nm                        | 1.0 %                        |
| 3F             | 730 nm                        | 16.7 %                       |
| 3A            | 681 nm                        | 3.5 %                        |
| 3B            | 704 nm                        | 3.4 %                        |
| 3C            | 727 nm                        | 2.8 %                        |
| 3G            | 752 nm                        | 3.4 %                        |
| 3D            | 754 nm                        | 1.0 %                        |
| 3E            | 778 nm                        | 3.5 %                        |
| 4F             | 825 nm                        | 11.4 %                       |
| 4A            | 792 nm                        | 3.5 %                        |
| 4B            | 825 nm                        | 3.6 %                        |
| 4C            | 857 nm                        | 3.5 %                        |


In [ ]:
bandpass = '1F'

Load a Pre-Saved DM File. Several pre-generated deformable mirror (DM) files are available for different coronagraph types and contrast configurations. The DM solution files are stored in the  ```examples ``` folder of  ```roman_preflight_proper ```

Available configurations:
- HLC:
    hlc_ni_2e-9, hlc_ni_3e-8, hlc_ni_5e-9

- SPC-Spec:
    spc-spec_ni_1e-9, spc-spec_ni_2e-8, spc-spec_ni_4e-9

- SPC-Wide:
    spc-wide_ni_2e-8, spc-wide_ni_3e-9, spc-wide_ni_5e-9

In [ ]:
cases = ['3e-8']       
rootname = 'hlc_ni_' + cases[0]
dm1 = proper.prop_fits_read( roman_preflight_proper.lib_dir + '/examples/'+rootname+'_dm1_v.fits' )
dm2 = proper.prop_fits_read( roman_preflight_proper.lib_dir + '/examples/'+rootname+'_dm2_v.fits' )


In [ ]:
##  Define the polaxis parameter. Use 10 for non-polaxis cases only, as other options are not yet implemented.
polaxis = 10
# output_dim define the size of the output image
output_dim = 51

### define a dictinatary to pass keywarod to proper
### proper_keywords are the keyword arguments to the internal functions for package proper, which define the optics of coronagraph
# use_dm1/use_dm2: if use dm
# use_fpm: if use focal plane mask
# use_lyot_stop: if use lyot stop 
# use_field_stop: if use field stop
# other paramters that could pass to Proper defined by CgiSim

proper_keywords ={'cor_type':cor_type, 'use_errors':2, 'polaxis':polaxis, 'output_dim':output_dim,\
                    'use_dm1':1, 'dm1_v':dm1, 'use_dm2':1, 'dm2_v':dm2,'use_fpm':1, 'use_lyot_stop':1,  'use_field_stop':1 }

##define the corgi.optics class that hold all information about the instrument paramters                    
optics = instrument.CorgiOptics(cgi_mode, bandpass, proper_keywords=proper_keywords, if_quiet=True)

In [ ]:
proper_keywords

Now let's plot the filter curve. 
Note that you don’t need to set up the bandpass manually when running CorgiSim — ```corgisim.optics``` will automatically select the appropriate bandpass based on the input filter number. The following is for illustration purposes only.

the output of ```optics.setup_bandpass``` is ```SpectralElement``` object from ```synphot```, details see https://synphot.readthedocs.io/en/latest/api/synphot.spectrum.SpectralElement.html#synphot.spectrum.SpectralElement

In [ ]:
nd_filter = 0
# bp = optics.setup_bandpass(cgi_mode, bandpass, nd_filter )
optics.bp.plot()

### 💡 Tip for Delevoper

If you want to integrate the stellar flux over a given bandpass, you can use the `Observation` class from `synphot`. This allows you to compute the photon count rate through a filter while accounting for the filter transmission curve. Below is an example of how to calculate the integrated photon counts for the stellar spectrum after applying a defined bandpass **bp** from the previous steps.


> ⚠️ You **do not** need to do this step manually for running corgisim.  
> This example is only meant to demonstrate how `corgisim` performs bandpass integration internally.



In [ ]:
from synphot import  Observation
# Compute the observed stellar spectrum within the defined bandpass
# obs: wavelegth is in unit of angstrom
# obs: flux is in unit of photons/s/cm^2/angstrom
obs = Observation(base_scene.stellar_spectrum, optics.bp)
#obs.plot()

area = 35895.212    # primary effective area for Roman from cgisim in cm^2
# Compute total photon counts integrated across the full bandpass
counts = obs.countrate(area=area)
print('Total counts across the bandpass:',counts)
## you can also integrate across a narrower wavelength range within the bandpass
counts_sub = obs.countrate(area=area,  waverange=[5700, 5800])
print('Total counts from 5700-5800 A:',counts_sub)

### Step 3: Generate the PSF for the on-axis star and inject companions

The `get_psf` function from the `CorgiOptics` class generates the on-axis PSF for the host star. It takes the following inputs:

In [ ]:
base_scene.twoD_scene_info

In [ ]:
## Pass the base_scene object to corgi.optics and use get_psf to simulate the host star PSF.
sim_scene = optics.get_host_star_psf(base_scene)
image_star_corgi = sim_scene.host_star_image.data

# simulate the planet PSF.
sim_scene = optics.inject_point_sources(base_scene,sim_scene)
image_comp_corgi = sim_scene.point_source_image.data 

In [ ]:
valid_positions = get_valid_polar_positions = conv.get_valid_polar_positions(radii_lamD, azimuths_deg)

# Unpack (r in λ/D, θ in deg)
res_mas = 49.3  # resolution at 550 nm in mas
pix_scale_mas = 0.0218*1000  # pixel scale in mas/pixel
r_lamD   = np.array([pos[0] for pos in valid_positions], dtype=float)
theta_deg = np.array([pos[1].value for pos in valid_positions], dtype=float)  # degrees

# Convert polar (λ/D, deg) -> Cartesian pixels
ny, nx = image_star_corgi.shape
cy, cx = ny // 2, nx // 2
r_pix = r_lamD * (res_mas / pix_scale_mas)
theta = np.deg2rad(theta_deg)

x = cx + r_pix * np.cos(theta)
y = cy + r_pix * np.sin(theta)

plt.imshow(image_star_corgi, origin='lower')
plt.scatter(x, y, s=10, label='Sample positions')
plt.legend(); plt.title('Sampling positions on PSF image')
plt.show()

### Step 4: 2D Scene Setup and Field-Dependent Convolution

The astrophysical scene has been initialized in `base_scene` using the configuration parameters defined above. This scene contains the host star, point source companions, and optionally a 2D extended structure (e.g., circumstellar disk). The next step involves performing field-dependent PSF convolution using pre-computed or dynamically generated PSF cubes.

#### Key Configuration Parameters

- **Radial Grid (`radii_lamD`)**: Defines the radial sampling points from the inner working angle to outer working angle. 
- **Azimuthal Grid (`azimuths_deg`)**: Azimuthal sampling for field-dependent PSF computation.
- **Disk Contrast**: Set to 16 delta magnitudes, representing the average surface brightness per resolution element relative to the stellar flux.
- **PSF Cube Path (`fname_prf`)**: Pre-computed PSF cube used for convolution operations.

#### Scene Components and Spectra

The `base_scene` object provides access to different spectral components:

```python
# Access individual spectral components
sp_disk = base_scene.twoD_scene_spectrum    # Extended disk spectrum
sp_star = base_scene.stellar_spectrum       # Host star spectrum  
sp_comp = base_scene.off_axis_source_spectrum # Companion spectrum(s)
```

#### Field-Dependent Convolution Workflow

1. **Scene Initialization**: The `base_scene` object contains the complete astrophysical model with host star, companions, and optional extended structures.

2. **PSF Grid Definition**: The radial and azimuthal grids (`radii_lamD`, `azimuths_deg`) define the field-dependent sampling for PSF convolution across the detector plane.

3. **Convolution Process**: The `convolution.py` module provides utilities for:
   - Field-dependent PSF interpolation based on the defined grids
   - Weighted convolution accounting for PSF variations across the field
   - Integration of multiple scene components (star, companions, disk)

4. **Output Generation**: After convolution, the processed scene will be available for further analysis, including:
   - Individual component images (star, companions, disk)
   - Combined scene image with all components
   - Spectral information preserved throughout the process


In the current implementation, there are two modes for performing convolution with the PSF:

#### **Mode 1: Using Precomputed PRF Cubes**
- Utilises precomputed PRF cubes stored .fits  
- matching radial/azimuthal grids --> information (e.g., `radii_lamD_1`, `azimuths_deg_1`) or at least we need to know the command generating it if using corgisim


#### **Mode 2: Generating PRF Cubes on the Fly**
- Dynamically generates PRF cubes, defines sampling points in the field of view (FoV).

#### Notes: Ensure grids match the PRF cube in **Mode 1**.

#### Important Notes

- **Disk Contrast Definition**: The 16 delta magnitude contrast represents the average surface brightness over disk regions containing >50% of maximum disk flux, not the entire field of view.
- **Grid Resolution**: The 1 λ/D radial step and 10° azimuthal step provide adequate sampling for most coronagraphic applications while maintaining computational efficiency.
- **PSF Cube Usage**: The pre-computed PSF cube (`fname_prf`) should match the instrument configuration and wavelength range of the current observation setup.

#### Next Steps

With the scene and convolution parameters configured, the next operations will involve:
- Executing the field-dependent convolution using the `convolution.py` utilities
- Generating the final convolved image products
- Analyzing the resulting coronagraphic image for scientific interpretation

In [ ]:
# simulate the disk
sim_scene = optics.convolve_2d_scene(input_scene=base_scene, sim_scene=sim_scene, use_bilinear_interpolation=True)
image_disk_corgi = sim_scene.twoD_image.data 

combined_image_corgi = image_star_corgi + image_disk_corgi + image_comp_corgi

In [ ]:
fig = plt.figure(figsize=(12+4,4))
plt.subplot(141)
plt.imshow(image_star_corgi,origin='lower')
plt.title('Host star Vmang=8, CorgiSim')

co = plt.colorbar(shrink=0.7)
plt.subplot(142)
plt.imshow(image_disk_corgi,origin='lower')
plt.title(r'Disk (2d scene; $\Delta$Mag=16 mag), CorgiSim')
co = plt.colorbar(shrink=0.7,)

plt.subplot(143)
plt.imshow(image_comp_corgi,origin='lower')
plt.title('Companion Vmag=25, CorgiSim')
co = plt.colorbar(shrink=0.7)

plt.subplot(144)
plt.imshow(combined_image_corgi,origin='lower')
plt.title('Combined Image, CorgiSim')

co = plt.colorbar(shrink=0.7)

#### Step 4.1: Build Sampling Grid for Field-Dependent PSF Computation

To perform field-dependent PSF computations, you need to define a sampling grid. This involves creating radial and azimuthal grids, which serve as the foundation for PSF sampling. Below is an example of how to use the `build_radial_grid` and `build_azimuth_grid` functions to construct these grids.

Offsets in the radial and azimuthal grids are essential for field-dependent PSF computations because the PSF (Point Spread Function) of an optical system can vary across the field of view. These variations arise due to:

1. **Field Dependence**:
    - Optical aberrations, such as coma and astigmatism, can cause the PSF to change depending on the position in the field.
    - The PSF at the center of the field (on-axis) is typically different from the PSF at the edges (off-axis).

2. **Sampling the Field**:
    - To accurately model and simulate the PSF across the field, we need to sample it at various positions. This is done by defining a grid of radial and azimuthal offsets.
    - The radial grid specifies distances from the optical axis (in units of λ/D), while the azimuthal grid specifies angular positions (in degrees).

3. **Interpolation**:
    - Once the PSF is computed or measured at these grid points, it can be interpolated to estimate the PSF at any arbitrary position within the field.
    - This ensures that the simulation accurately represents the spatially varying PSF.

4. **Convolution with the Scene**:
    - The offsets allow the PSF to be applied correctly to different parts of the astrophysical scene during convolution, ensuring that the final image accounts for field-dependent effects.

In summary, these offsets are critical for capturing the spatial variations in the PSF and ensuring that the simulated images are realistic and accurate. Without these offsets, the simulation would assume a uniform PSF across the field, which is not representative of real optical systems.

Below is an example of how to define the radial and azimuthal grids using the `build_radial_grid` and `build_azimuth_grid` functions:


In [ ]:
from corgisim import convolution as conv
import time  
import sys
import corgisim.convolution as conv 

# radial: 0, 5, 10  λ/D   (3 nodes)
# azimuth: 0–270° in steps of 90° (4 nodes)

radii_lamD_1 = conv.build_radial_grid(
    inner_step      = 5,     # step inside IWA (λ/D)
    mid_step        = 5,     # step between IWA and OWA
    outer_step      = 5,     # step beyond OWA (λ/D)
    iwa             = 0,     # inner working angle (λ/D)
    owa             = 10,    # outer working angle (λ/D)
)

azimuths_deg_1 = conv.build_azimuth_grid(
    step_deg        = 90  # azimuthal step in degrees
)



### Step 4.3 (Optional): Creating a Small Cube for Faster Processing

To speed up the processing, we will create a smaller cube. This smaller cube will be used for testing and debugging purposes, ensuring quicker iterations while maintaining the core functionality.


In [ ]:
test_prf = optics.make_prf_cube(radii_lamD_1, azimuths_deg_1)

In [ ]:
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np

# Create a figure and axis for the animation
fig, ax = plt.subplots()
im = ax.imshow(test_prf[0], cmap='viridis', norm=LogNorm(vmin=test_prf.min(), vmax=test_prf.max()))
ax.axis('off')
ax.set_title("PRF Changes Over Layers")

# Update function for the animation
def update(frame):
    im.set_data(test_prf[frame])
    ax.set_title(f"PRF Layer {frame + 1}/{test_prf.shape[0]}")
    return [im]

# Create the animation
ani = FuncAnimation(fig, update, frames=test_prf.shape[0], interval=200, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())

#### mode 2: Generate PRF cubes on the fly


In [ ]:
twoD_scene_info = {
        'disk_model_path': fname_disk,
        'contrast': disk_contrast, 
        'prf_cube_path': None,
        'radii_lamD': None,
        'azimuths_deg': None
    }

In [ ]:
# --- Create the Astrophysical Scene ---
# This Scene object combines the host star and companion(s), and optionally a 2D scene (e.g., disk)
base_scene_2 = scene.Scene(host_star_properties, point_source_info, twoD_scene_info)

In [ ]:
sim_scene_2 = optics.convolve_2d_scene(base_scene_2,iwa=0, owa=10, inner_step=5, mid_step=5, outer_step=5, step_deg=90)

In [ ]:
plt.imshow(sim_scene_2.twoD_image.data)

### Step 5: Simulate the Image on the Detector

So far, we've simulated images of a host star with two companions using Roman-CGI. However, we haven't yet included detector noise! In this step, we'll define a **`corgi.Detector`** object to add detector effects to the simulation.

The `emccd_keywords` dictionary is passed to `instrument.CorgiDetector` to configure the EMCCD detector. It includes the following parameters:

- **em_gain**: Electron multiplication gain. Default is `1000`.
- **full_well_image**: Full well capacity for the image section. Requirement: `50,000`; CBE: `60,000`.
- **full_well_serial**: Full well capacity for the serial register. Requirement: `90,000`; CBE: `100,000`.
- **dark_rate**: Dark current rate in e⁻/pix/s. Requirement: `1.0`; CBE: `0.00042` (0 yrs) / `0.00056` (5 yrs).
- **cic_noise**: Clock-induced charge noise in e⁻/pix/frame. Default: `0.01`.
- **read_noise**: Read noise in e⁻/pix/frame. Requirement: `125`; CBE: `100`.
- **bias**: Bias level in digital numbers (DN). Default: `0`.
- **qe**: Quantum efficiency. Set to `1.0` here, since it's already factored into the photon counts.
- **cr_rate**: Cosmic ray event rate in hits/cm²/s. Use `0` for no cosmic rays, `5` for L2 environment.
- **pixel_pitch**: Pixel pitch in meters. Default: `13e-6`.
- **e_per_dn**: Electrons per data unit after multiplication.
- **numel_gain_register**: Number of elements in the gain register. Default: `604`.
- **nbits**: Number of bits in the ADC (analog-to-digital converter). Default: `14`.
- **use_traps**: Whether to simulate CTI effects using trap models. Default: `False`.
- **date4traps**: Decimal year of observation. Only used if `use_traps=True`. Default: `2028.0`.


In [ ]:
### emccd_keywords are the keyword arguments to the internal functions for  emccd_detect,
###  and that everything stays the default settings unless otherwise changed.
### In this example, we'll use the default parameters for the EMCCD detector, except for the EM gain.

gain =100
emccd_keywords ={'em_gain':gain}
detector = instrument.CorgiDetector(emccd_keywords)


The `generate_detector_image` function from the `CorgiDetector` class generates a detector image from the simulated input scene. It takes the following arguments:

- **simulated_scene**: A `corgisim.scene.SimulatedScene` object that contains the noise-free scene from CorgiOptics
- **exptime**: Exposure time in seconds.
- **full_frame** *(bool)*: If `True`, generates a full-frame detector image and places the sub-frame within it.
- **loc_x** *(int)*: Horizontal (x) pixel coordinate of the center where the sub-frame will be inserted. Required if `full_frame=True`.
- **loc_y** *(int)*: Vertical (y) pixel coordinate of the center where the sub-frame will be inserted. Required if `full_frame=True`.


In [ ]:

# In real observations, exposures are typically broken into a sequence of short frames (e.g., 100s per frame) to reduce the impact of cosmic ray hits.
# The multi-exposure simulation will be shown below.

exptime = 20
sim_scene = detector.generate_detector_image(sim_scene,exptime)
image_tot_corgi_sub= sim_scene.image_on_detector.data

In [ ]:
plt.imshow(image_tot_corgi_sub,origin='lower')
plt.title('Combined Image with detector noise (single short exposure), CorgiSim')

co = plt.colorbar(shrink=0.7)

We're not done yet! To complete the simulation, we need to generate a **Level 1 (L1) data product** — a full-frame EMCCD image.

This requires placing the subframe we just simulated onto the full Roman EMCCD frame.  
You can do this by setting the keyword `full_frame=True` in the detector call.

- `loc_x` and `loc_y` specify the center position (in pixels) where the subframe should be placed on the full EMCCD frame.


In [ ]:
sim_scene = detector.generate_detector_image(sim_scene,exptime,full_frame=True,loc_x=300, loc_y=300)
image_tot_corgi_full = sim_scene.image_on_detector[1].data

In [ ]:
plt.imshow(image_tot_corgi_full,origin='lower')
plt.title('Combined Image full frame (single exp), CorgiSim')

co = plt.colorbar(shrink=0.7)

To aviod the impact of cosmic ray hits, here we simulate multiple exposures and median combine image.

In [ ]:
gain =100
emccd_keywords ={'em_gain':gain}
detector = instrument.CorgiDetector(emccd_keywords)


exptime = 100 # intergration time per exposure (s) 
num_exp = 100 # number of exposures per observation
output = np.zeros((num_exp, image_tot_corgi_sub.shape[0], image_tot_corgi_sub.shape[1]))
for t in range(num_exp):
    sim_scene = detector.generate_detector_image(sim_scene,exptime)
    image_tot_corgi_sub= sim_scene.image_on_detector.data
    output[t] = image_tot_corgi_sub


plt.imshow(np.nanmedian(output, axis=0),origin='lower')
plt.title('Combined Image with detector noise (multi exp: {0}s X {1}), CorgiSim'.format(exptime, num_exp))
plt.colorbar()

In [ ]:
num_exp = 100 # number of exposures per observation
output_full = np.zeros((num_exp, image_tot_corgi_full.shape[0], image_tot_corgi_full.shape[1]))
for t in range(num_exp):
    sim_scene = detector.generate_detector_image(sim_scene,exptime, full_frame=True,loc_x=300, loc_y=300)
    image_tot_corgi_full = sim_scene.image_on_detector[1].data
    output_full[t] = image_tot_corgi_full

plt.imshow(np.nanmedian(output_full, axis=0),origin='lower')
plt.title('Combined Image full frame (multi exp combined), CorgiSim')

co = plt.colorbar(shrink=0.7)

In [ ]:
##you can set your output path here
## If outdir is None, the output fits files will be saved in the current working directory.
outdir = None
### save intemediate products
outputs.save_hdu_to_fits(sim_scene.host_star_image, outdir = outdir, filename='host_star_image', write_as_L1=False)

### save final L1 producr
outputs.save_hdu_to_fits(sim_scene.image_on_detector, outdir = outdir, write_as_L1=True)
   